In [1]:
import sys
import os
myDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(myDir, os.pardir))
sys.path.append(parentDir)

from pathlib import Path
path = Path(parentDir)
a=str(path.parent.absolute())

sys.path.append(a)

In [2]:
from MetaX.utils.taxaFuncAnalyzer import TaxaFuncAnalyzer
from MetaX.utils.taxaFuncPloter.basic_plot import BasicPlot


In [3]:
df_path = 'example_data/SW_TaxaFunc.tsv'
meta_path = 'example_data/SW_meta.tsv'
sw = TaxaFuncAnalyzer(df_path, meta_path)

In [13]:
sw.set_func('Description')
sw.set_multi_tables(level='s')

sw.set_group('Person')

batch_list is not set, Batch effect removal did not perform.
transform_method is not set, data transform did not perform.
normalize_method is not set, data normalization did not perform.
Original data shape: (3999, 229)
Starting to set Function table...
Function number: 830
Starting to set Taxa table...
Taxa number: 340
Taxa-Function number: 1272
group is set to Person
 {'V3', 'V4', 'V5', 'V2', 'V1'}


In [14]:
df_fc = sw.get_stats_deseq2(df=sw.taxa_df,group_list=['V1', 'V2'])

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.17 seconds.

Fitting dispersion trend curve...
... done in 0.14 seconds.

Fitting MAP dispersions...
... done in 0.17 seconds.

Fitting LFCs...
... done in 0.27 seconds.

Refitting 97 outliers.

Fitting dispersions...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.06 seconds.

Running Wald tests...
... done in 0.16 seconds.

Log2 fold change & Wald test p-value: Person V2 vs V1


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Taxon,,,,,,
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis,768.559077,-11.849081,0.000000,0.000000,NaN,NaN
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve,865.281980,20.426810,7.704176,2.651395,8.016015e-03,1.825075e-02
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis,25083.403224,2.300576,0.471860,4.875548,1.085069e-06,4.611542e-06
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium pseudocatenulatum,53207.583866,1.837699,0.217110,8.464351,2.575827e-17,2.345842e-16
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium ruminantium,651.309392,43.280851,49.788335,0.869297,3.846847e-01,5.109094e-01
...,...,...,...,...,...,...
d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A,681.295537,-28.940109,155.761259,-0.185798,8.526033e-01,9.212451e-01
d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Serratia|s__Serratia ureilytica,1142.133966,-0.623882,2.885393,-0.216221,8.288156e-01,9.070729e-01
d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Succinivibrionaceae|g__Anaerobiospirillum|s__Anaerobiospirillum sp900543125,2331.585369,-13.465430,0.000000,0.000000,NaN,NaN


In [26]:

from pyecharts.charts import Scatter
from pyecharts import options as opts
import numpy as np


class VolcanoPlot():


    def plot_volcano_js(self, df_fc, padj:float=0.05, log2fc_min:float=1, log2fc_max:float = 10, title_name:str='2 groups', width:int=1200, height:int=800):
        df = df_fc.copy()
       
        
        df.loc[(df['padj'] < padj) & (
            df['log2FoldChange'] > log2fc_min) & (df['log2FoldChange'] < log2fc_max) , 'type'] = 'up'
        
        df.loc[(df['padj'] < padj) & (
            df['log2FoldChange'] > log2fc_max) , 'type'] = 'ultra-up'
        
        df.loc[(df['padj'] < padj) & (
            df['log2FoldChange'] < -log2fc_min) & (df['log2FoldChange'] > -log2fc_max) , 'type'] = 'down'
        
        df.loc[(df['padj'] < padj) & (
            df['log2FoldChange'] < -log2fc_max) , 'type'] = 'ultra-down'
        
        df.loc[df['type'].isnull(), 'type'] = 'normal'

        count_dict = {}
        for i in ['up', 'down', 'ultra-up', 'ultra-down', 'normal']:
            count_dict[i] = len(df[df['type'] == i])    
        
       
        count_normal = len(df[df['type'] == 'normal'])

        # create a new column for label
        df['label'] = df.index
        # extract the columns we need
        df = df[['log2FoldChange', 'padj', 'label', 'type']]
        # -log10(padj)
        df['padj'].fillna(1, inplace=True)
        df['padj'] = df['padj'].apply(lambda x: -np.log10(x))
        df['log2FoldChange'] = df['log2FoldChange'].apply(lambda x: round(x, 3))
        df['padj'] = df['padj'].apply(lambda x: round(x, 3))
        
        def color_mapping(type_value):
            if type_value == 'up':
                return "#d23918"
            elif type_value == 'down':
                return "#68945c"
            elif type_value == 'ultra-up':
                return "#663d74"
            elif type_value == 'ultra-down':
                return "#206864"
            else:
                return "#9aa7b1"

        # 创建一个新的列'itemStyle'，其值是一个 ItemStyleOpts 对象
        df['itemStyle'] = df['type'].apply(lambda x: opts.ItemStyleOpts(color=color_mapping(x)))

        Scatter_up = df[df['type'] == 'up'].apply(lambda p: {'name': p['label'], 'value': [p['log2FoldChange'], p['padj']], 'itemStyle': p['itemStyle']}, axis=1)
        scatter_ultra_up = df[df['type'] == 'ultra-up'].apply(lambda p: {'name': p['label'], 'value': [p['log2FoldChange'], p['padj']], 'itemStyle': p['itemStyle']}, axis=1)
        Scatter_down = df[df['type'] == 'down'].apply(lambda p: {'name': p['label'], 'value': [p['log2FoldChange'], p['padj']], 'itemStyle': p['itemStyle']}, axis=1)
        scatter_ultra_down = df[df['type'] == 'ultra-down'].apply(lambda p: {'name': p['label'], 'value': [p['log2FoldChange'], p['padj']], 'itemStyle': p['itemStyle']}, axis=1)
        Scatter_normal = df[df['type'] == 'normal'].apply(lambda p: {'name': p['label'], 'value': [p['log2FoldChange'], p['padj']], 'itemStyle': p['itemStyle']}, axis=1)

        title = f'Volcano plot of {title_name} (padj < {padj}, log2FoldChange > {log2fc_min})'
        scatter = (
            Scatter(init_opts=opts.InitOpts(width=f"{width}px", height=f"{height}px"))
            .add_xaxis(df['log2FoldChange'].tolist())
            .add_yaxis(
                f"Normal ({count_dict['normal']})",
                Scatter_normal.tolist(),
                label_opts=opts.LabelOpts(is_show=False),
            )
            .set_global_opts(
                xaxis_opts=opts.AxisOpts(type_="value", splitline_opts=opts.SplitLineOpts(is_show=False), name='log2FoldChange'),
                yaxis_opts=opts.AxisOpts(type_="value", splitline_opts=opts.SplitLineOpts(is_show=False), name='-log10(padj)'),
                title_opts=opts.TitleOpts(title=title, pos_left='center'),
                legend_opts=opts.LegendOpts(pos_left='right',  orient='vertical'),
                tooltip_opts=opts.TooltipOpts(is_show=True, position='top', formatter='{b}: {c}'),
                toolbox_opts=opts.ToolboxOpts( is_show=True, feature={"saveAsImage": {}, "restore": {}, "dataZoom": {}}),
            ) )
        
        if count_dict['up'] > 0:
            scatter.add_yaxis(
                f"Up ({count_dict['up']})",
                Scatter_up.tolist(),
                label_opts=opts.LabelOpts(is_show=False),
            )
        if count_dict['down'] > 0:
            scatter.add_yaxis(
                f"Down ({count_dict['down']})",
                Scatter_down.tolist(),
                label_opts=opts.LabelOpts(is_show=False) )
        if count_dict['ultra-up'] > 0:
            scatter.add_yaxis(
                f"Ultra-up ({count_dict['ultra-up']})",
                scatter_ultra_up.tolist(),
                label_opts=opts.LabelOpts(is_show=False) )
        if count_dict['ultra-down'] > 0:
            scatter.add_yaxis(
                f"Ultra-down ({count_dict['ultra-down']})",
                scatter_ultra_down.tolist(),
                label_opts=opts.LabelOpts(is_show=False) )
        
        
            
        return scatter

VolcanoPlot().plot_volcano_js(df_fc, padj=0.05, log2fc_min=1,log2fc_max=15, title_name='2 groups', width=1200, height=800).render_notebook()